In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score
import glob

In [41]:
class DACNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DACNN, self).__init__()

        # 根据表格构建网络结构
        # 第1层: Conv1 + MaxPool
        self.shared_conv1 = nn.Conv1d(1, 8, kernel_size=32, stride=2)  # 输出: 1009×8
        self.shared_pool1 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 504×8

        # 第2层: Conv2 + MaxPool
        self.shared_conv2 = nn.Conv1d(8, 16, kernel_size=16, stride=2)  # 输出: 245×16
        self.shared_pool2 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 122×16

        # 第3层: Conv3 + MaxPool
        self.shared_conv3 = nn.Conv1d(16, 32, kernel_size=8, stride=2)  # 输出: 58×32
        self.shared_pool3 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 29×32

        # 第4层: 源域和目标域特定的卷积层
        self.source_conv4 = nn.Conv1d(32, 32, kernel_size=8, stride=2)  # 输出: 29×32
        self.source_pool4 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 14×32

        self.target_conv4 = nn.Conv1d(32, 32, kernel_size=8, stride=2)  # 输出: 29×32
        self.target_pool4 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 14×32

        # 第5层: 源域和目标域特定的卷积层
        self.source_conv5 = nn.Conv1d(32, 64, kernel_size=3, stride=2)  # 输出: 2×64
        self.source_pool5 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 1×64

        self.target_conv5 = nn.Conv1d(32, 64, kernel_size=3, stride=2)  # 输出: 2×64
        self.target_pool5 = nn.MaxPool1d(kernel_size=2, stride=2)  # 输出: 1×64

        # 全连接层
        self.shared_fc1 = nn.Linear(64, 500)  # 输入: 1×64 = 64
        self.source_fc2 = nn.Linear(500, num_classes)
        self.target_fc2 = nn.Linear(500, num_classes)

        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def source_forward(self, x):
        # 共享层
        x = self.relu(self.shared_conv1(x))
        x = self.shared_pool1(x)

        x = self.relu(self.shared_conv2(x))
        x = self.shared_pool2(x)

        x = self.relu(self.shared_conv3(x))
        x = self.shared_pool3(x)

        # 源域特定层
        x = self.relu(self.source_conv4(x))
        x = self.source_pool4(x)

        x = self.relu(self.source_conv5(x))
        x = self.source_pool5(x)

        # 全连接层
        x = x.view(x.size(0), -1)
        x = self.relu(self.shared_fc1(x))
        x = self.dropout(x)
        x = self.source_fc2(x)

        return x

    def target_forward(self, x):
        # 共享层
        x = self.relu(self.shared_conv1(x))
        x = self.shared_pool1(x)

        x = self.relu(self.shared_conv2(x))
        x = self.shared_pool2(x)

        x = self.relu(self.shared_conv3(x))
        x = self.shared_pool3(x)

        # 目标域特定层
        x = self.relu(self.target_conv4(x))
        x = self.target_pool4(x)

        x = self.relu(self.target_conv5(x))
        x = self.target_pool5(x)

        # 全连接层
        x = x.view(x.size(0), -1)
        x = self.relu(self.shared_fc1(x))
        x = self.dropout(x)
        x = self.target_fc2(x)

        return x

    def source_features(self, x):
        # 获取源域特征（用于MMD计算）
        x = self.relu(self.shared_conv1(x))
        x = self.shared_pool1(x)

        x = self.relu(self.shared_conv2(x))
        x = self.shared_pool2(x)

        x = self.relu(self.shared_conv3(x))
        x = self.shared_pool3(x)

        x = self.relu(self.source_conv4(x))
        x = self.source_pool4(x)

        x = self.relu(self.source_conv5(x))
        x = self.source_pool5(x)

        x = x.view(x.size(0), -1)
        features = self.relu(self.shared_fc1(x))

        return features

    def target_features(self, x):
        # 获取目标域特征（用于MMD计算）
        x = self.relu(self.shared_conv1(x))
        x = self.shared_pool1(x)

        x = self.relu(self.shared_conv2(x))
        x = self.shared_pool2(x)

        x = self.relu(self.shared_conv3(x))
        x = self.shared_pool3(x)

        x = self.relu(self.target_conv4(x))
        x = self.target_pool4(x)

        x = self.relu(self.target_conv5(x))
        x = self.target_pool5(x)

        x = x.view(x.size(0), -1)
        features = self.relu(self.shared_fc1(x))

        return features

In [42]:
class MMDLoss(nn.Module):
    def __init__(self, kernel_mul=2.0, kernel_num=5):
        super(MMDLoss, self).__init__()
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        self.fix_sigma = None

    def guassian_kernel(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
        n_samples = int(source.size()[0]) + int(target.size()[0])
        total = torch.cat([source, target], dim=0)
        total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_distance = ((total0 - total1) ** 2).sum(2)
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            bandwidth = torch.sum(L2_distance.data) / (n_samples ** 2 - n_samples)
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul ** i) for i in range(kernel_num)]
        kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
        return sum(kernel_val)

    def forward(self, source, target):
        batch_size = int(source.size()[0])
        kernels = self.guassian_kernel(source, target, kernel_mul=self.kernel_mul,
                                     kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
        XX = kernels[:batch_size, :batch_size]
        YY = kernels[batch_size:, batch_size:]
        XY = kernels[:batch_size, batch_size:]
        YX = kernels[batch_size:, :batch_size]
        loss = torch.mean(XX + YY - XY - YX)
        return loss

In [43]:
class CWRUDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [51]:
def load_data_from_structure(data_path, load_conditions=['1', '2', '3']):
    """从您提供的目录结构加载数据"""

    # 类别映射：10个类别（9种故障 + 1种正常）
    # 类别0: 正常 (N)
    # 类别1: IF_0.007
    # 类别2: IF_0.014
    # 类别3: IF_0.021
    # 类别4: BF_0.007
    # 类别5: BF_0.014
    # 类别6: BF_0.021
    # 类别7: OF_0.007
    # 类别8: OF_0.014
    # 类别9: OF_0.021

    domains = {}

    for load_condition in load_conditions:
        domain_data = []
        domain_labels = []
        load_path = os.path.join(data_path, load_condition)

        if not os.path.exists(load_path):
            print(f"Warning: Load condition {load_condition} path {load_path} does not exist")
            continue

        print(f"Processing load condition {load_condition}...")

        # 类别0: 正常状态
        normal_file = os.path.join(load_path, 'N.mat')
        if os.path.exists(normal_file):
            print(f"  Processing normal condition...")
            samples = process_mat_file(normal_file, 800, 0)
            domain_data.extend(samples)
            domain_labels.extend([0] * len(samples))

        # 故障类型和对应的尺寸
        fault_types = ['IF', 'BF', 'OF']
        fault_sizes = ['7', '14', '21']

        label_counter = 1
        for fault_type in fault_types:
            for fault_size in fault_sizes:
                fault_dir = os.path.join(load_path, fault_size)
                if not os.path.exists(fault_dir):
                    continue

                # 查找对应的故障文件
                fault_pattern = os.path.join(fault_dir, f'{fault_type}*.mat')
                fault_files = glob.glob(fault_pattern)

                if fault_files:
                    # 取第一个匹配的文件
                    fault_file = fault_files[0]
                    print(f"  Processing {fault_type}_{fault_size}...")
                    samples = process_mat_file(fault_file, 800, label_counter)
                    domain_data.extend(samples)
                    domain_labels.extend([label_counter] * len(samples))

                label_counter += 1

        if domain_data:
            domains[load_condition] = {
                'data': np.array(domain_data, dtype=np.float32),
                'labels': np.array(domain_labels, dtype=np.int64)
            }
            print(f"Domain {load_condition}: {len(domain_data)} samples")
        else:
            print(f"Warning: No data found for domain {load_condition}")

    return domains

def process_mat_file(file_path, num_samples, label):
    """处理单个.mat文件，生成指定数量的样本"""
    try:
        # 加载MAT文件
        mat_data = sio.loadmat(file_path)

        # 找到包含振动数据的变量（排除元数据变量）
        data_key = None
        for key in mat_data.keys():
            if not key.startswith('__') and not key.startswith('header') and not key.startswith('version'):
                # 检查是否是数值数组
                data = mat_data[key]
                if isinstance(data, np.ndarray) and data.size > 1000:
                    data_key = key
                    break

        if data_key is None:
            print(f"Warning: No suitable data found in {file_path}")
            # 生成合成数据用于测试
            vibration_data = np.random.randn(100000)
        else:
            vibration_data = mat_data[data_key].flatten()

        samples = []
        segment_length = 4096

        # 确保有足够的数据
        if len(vibration_data) < segment_length:
            print(f"Warning: Data in {file_path} is too short ({len(vibration_data)} < {segment_length})")
            # 重复数据以达到所需长度
            repeats = (segment_length // len(vibration_data)) + 1
            vibration_data = np.tile(vibration_data, repeats)

        for i in range(num_samples):
            # 随机选择起始位置
            start_idx = np.random.randint(0, len(vibration_data) - segment_length)
            segment = vibration_data[start_idx:start_idx + segment_length]

            # FFT变换
            fft_segment = np.fft.fft(segment)
            fft_magnitude = np.abs(fft_segment)[:2048]  # 取前2048个系数

            # 归一化
            fft_magnitude = (fft_magnitude - np.mean(fft_magnitude)) / (np.std(fft_magnitude) + 1e-8)

            samples.append(fft_magnitude)

        return samples

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        # 生成合成数据
        print("Generating synthetic data for testing...")
        samples = []
        for i in range(num_samples):
            synthetic_fft = np.random.randn(2048)
            samples.append(synthetic_fft)
        return samples


In [52]:
def pretrain_model(model, source_dataloader, device, num_epochs=100):
    """预训练阶段"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0

        for batch_data, batch_labels in source_dataloader:
            batch_data = batch_data.unsqueeze(1).to(device)  # 添加channel维度
            batch_labels = batch_labels.to(device)

            optimizer.zero_grad()
            outputs = model.source_forward(batch_data)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()

        if (epoch + 1) % 2 == 0:
            accuracy = 100 * correct / total
            print(f'Pre-train Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(source_dataloader):.4f}, Accuracy: {accuracy:.2f}%')

    return model

def domain_adaptive_finetune(model, source_dataloader, target_dataloader, device, num_epochs=100, lambda_mmd=1.0):
    """域自适应微调阶段"""
    criterion_cls = nn.CrossEntropyLoss()
    criterion_mmd = MMDLoss()

    # 只优化目标域特定的参数
    target_params = list(model.target_conv4.parameters()) + list(model.target_conv5.parameters()) + \
                   list(model.target_fc2.parameters())
    optimizer = optim.Adam(target_params, lr=0.0001)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        total_cls_loss = 0
        total_mmd_loss = 0

        source_iter = iter(source_dataloader)
        target_iter = iter(target_dataloader)

        num_batches = min(len(source_dataloader), len(target_dataloader))

        for batch_idx in range(num_batches):
            try:
                # 获取源域和目标域批次数据
                source_data, source_labels = next(source_iter)
                target_data, _ = next(target_iter)

                source_data = source_data.unsqueeze(1).to(device)
                source_labels = source_labels.to(device)
                target_data = target_data.unsqueeze(1).to(device)

                optimizer.zero_grad()

                # 源域分类损失
                source_outputs = model.source_forward(source_data)
                cls_loss = criterion_cls(source_outputs, source_labels)

                # MMD损失
                source_features = model.source_features(source_data)
                target_features = model.target_features(target_data)
                mmd_loss = criterion_mmd(source_features, target_features)

                # 总损失
                loss = cls_loss + lambda_mmd * mmd_loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                total_cls_loss += cls_loss.item()
                total_mmd_loss += mmd_loss.item()

            except StopIteration:
                break

        if (epoch + 1) % 20 == 0:
            avg_loss = total_loss / num_batches
            avg_cls_loss = total_cls_loss / num_batches
            avg_mmd_loss = total_mmd_loss / num_batches
            print(f'Fine-tune Epoch [{epoch+1}/{num_epochs}], Total Loss: {avg_loss:.4f}, '
                  f'Cls Loss: {avg_cls_loss:.4f}, MMD Loss: {avg_mmd_loss:.4f}')

    return model

In [46]:
def evaluate_model(model, dataloader, device, domain='target'):
    """评估模型性能"""
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch_data, batch_labels in dataloader:
            batch_data = batch_data.unsqueeze(1).to(device)

            if domain == 'target':
                outputs = model.target_forward(batch_data)
            else:
                outputs = model.source_forward(batch_data)

            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_predictions, average='macro', zero_division=0)

    return accuracy, precision, recall, all_predictions, all_labels

In [53]:
def main():
    # 参数设置
    data_path = r"D:\deskbook\科研\数据集\cwru\data"  # 修改为实际数据路径
    batch_size = 64
    num_epochs_pretrain = 100
    num_epochs_finetune = 100
    lambda_mmd = 1.0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"Using device: {device}")

    # 加载数据（只使用负载1、2、3）
    print("Loading data from directory structure...")
    domains = load_data_from_structure(data_path, load_conditions=['1', '2', '3'])

    # 定义域适配任务（例如 1->2）
    source_domain = '2'
    target_domain = '3'

    if source_domain not in domains or target_domain not in domains:
        print(f"Error: Required domains not found. Available domains: {list(domains.keys())}")
        return

    source_data = domains[source_domain]['data']
    source_labels = domains[source_domain]['labels']
    target_data = domains[target_domain]['data']
    target_labels = domains[target_domain]['labels']

    print(f"Source domain {source_domain}: {len(source_data)} samples")
    print(f"Target domain {target_domain}: {len(target_data)} samples")

    # 创建数据加载器
    source_dataset = CWRUDataset(source_data, source_labels)
    target_dataset = CWRUDataset(target_data, target_labels)

    source_dataloader = DataLoader(source_dataset, batch_size=batch_size, shuffle=True)
    target_dataloader = DataLoader(target_dataset, batch_size=batch_size, shuffle=True)

    # 创建模型
    model = DACNN(num_classes=10).to(device)

    # 阶段1: 预训练
    print("Stage 1: Pre-training on source domain...")
    model = pretrain_model(model, source_dataloader, device, num_epochs_pretrain)

    # 评估预训练模型在源域和目标域的性能
    print("\nEvaluating pre-trained model...")
    source_accuracy, source_precision, source_recall, _, _ = evaluate_model(
        model, DataLoader(source_dataset, batch_size=batch_size, shuffle=False), device, 'source')

    target_accuracy, target_precision, target_recall, _, _ = evaluate_model(
        model, DataLoader(target_dataset, batch_size=batch_size, shuffle=False), device, 'source')

    print(f"Pre-trained model - Source Domain: Accuracy: {source_accuracy:.4f}, "
          f"Precision: {source_precision:.4f}, Recall: {source_recall:.4f}")
    print(f"Pre-trained model - Target Domain: Accuracy: {target_accuracy:.4f}, "
          f"Precision: {target_precision:.4f}, Recall: {target_recall:.4f}")

    # 阶段2: 域自适应微调
    print("\nStage 2: Domain adaptive fine-tuning...")
    model = domain_adaptive_finetune(model, source_dataloader, target_dataloader,
                                   device, num_epochs_finetune, lambda_mmd)

    # 评估最终模型
    print("\nEvaluating fine-tuned model...")
    final_accuracy, final_precision, final_recall, predictions, labels = evaluate_model(
        model, DataLoader(target_dataset, batch_size=batch_size, shuffle=False), device, 'target')

    print(f"Fine-tuned model - Target Domain: Accuracy: {final_accuracy:.4f}, "
          f"Precision: {final_precision:.4f}, Recall: {final_recall:.4f}")

    # 保存模型
    torch.save(model.state_dict(), f'dacnn_model_{source_domain}_to_{target_domain}.pth')
    print(f"Model saved as dacnn_model_{source_domain}_to_{target_domain}.pth")

if __name__ == "__main__":
    main()

Using device: cuda
Loading data from directory structure...
Processing load condition 1...
  Processing normal condition...
  Processing IF_7...
  Processing IF_14...
  Processing IF_21...
  Processing BF_7...
  Processing BF_14...
  Processing BF_21...
  Processing OF_7...
  Processing OF_14...
  Processing OF_21...
Domain 1: 8000 samples
Processing load condition 2...
  Processing normal condition...
  Processing IF_7...
  Processing IF_14...
  Processing IF_21...
  Processing BF_7...
  Processing BF_14...
  Processing BF_21...
  Processing OF_7...
  Processing OF_14...
  Processing OF_21...
Domain 2: 8000 samples
Processing load condition 3...
  Processing normal condition...
  Processing IF_7...
  Processing IF_14...
  Processing IF_21...
  Processing BF_7...
  Processing BF_14...
  Processing BF_21...
  Processing OF_7...
  Processing OF_14...
  Processing OF_21...
Domain 3: 8000 samples
Source domain 2: 8000 samples
Target domain 3: 8000 samples
Stage 1: Pre-training on source do

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary


# ===== 改进后的 DACNN 模型 =====
class DACNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DACNN, self).__init__()

        # 共享卷积部分
        self.shared_conv = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=32, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),

            nn.Conv1d(8, 16, kernel_size=16, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),

            nn.Conv1d(16, 32, kernel_size=8, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
        )

        # 源域和目标域特有卷积部分
        self.source_branch = nn.Sequential(
            nn.Conv1d(32, 32, kernel_size=8, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),

            nn.Conv1d(32, 64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
        )

        self.target_branch = nn.Sequential(
            nn.Conv1d(32, 32, kernel_size=8, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),

            nn.Conv1d(32, 64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool1d(2, 2),
        )

        # 全连接部分
        self.fc_shared = nn.Sequential(
            nn.Linear(64, 500),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.fc_source = nn.Linear(500, num_classes)
        self.fc_target = nn.Linear(500, num_classes)

    # ===== 源域前向 =====
    def source_forward(self, x):
        x = self.shared_conv(x)
        x = self.source_branch(x)
        x = x.view(x.size(0), -1)
        x = self.fc_shared(x)
        out = self.fc_source(x)
        return out

    # ===== 目标域前向 =====
    def target_forward(self, x):
        x = self.shared_conv(x)
        x = self.target_branch(x)
        x = x.view(x.size(0), -1)
        x = self.fc_shared(x)
        out = self.fc_target(x)
        return out

    # ===== 提取特征用于 MMD =====
    def extract_features(self, x, domain='source'):
        x = self.shared_conv(x)
        x = self.source_branch(x) if domain == 'source' else self.target_branch(x)
        x = x.view(x.size(0), -1)
        features = self.fc_shared[0:2](x)  # 仅取到 ReLU，不包括 Dropout
        return features

    # ===== 用于 summary 的统一 forward（默认 source）=====
    def forward(self, x, domain='source'):
        if domain == 'target':
            return self.target_forward(x)
        return self.source_forward(x)


# 测试模型结构
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DACNN(num_classes=10).to(device)
summary(model, input_size=(1, 2048))
